<a href="https://colab.research.google.com/github/salah-2024-analyticshub/binary-text-classification-compare-models/blob/main/ML_Model_Compare_NLP_Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import numpy as np
import cProfile
import pstats
import io
import os
import psutil
import re
import random
import json
from sklearn.utils import resample
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import joblib
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from timeit import default_timer as timer
from datetime import timedelta
from datetime import datetime

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
dataset = pd.read_csv("/content/drive/MyDrive/imdb_reviews.csv")
dataset = dataset.rename(columns={'review': 'sentence', 'sentiment': 'flag'})
dataset['flag'] = dataset['flag'].str.replace('positive', '1')
dataset['flag'] = dataset['flag'].str.replace('negative', '0')
dataset.head()

,sentence,flag
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
model_pipeline1 = Pipeline([
          ('tfidf',HashingVectorizer(token_pattern=r'\w{3,}', stop_words='english', alternate_sign = False)),
          ('clf', OneVsRestClassifier(LogisticRegression(solver='sag',max_iter=500), n_jobs=1)),])

model_pipeline2 = Pipeline([
           ('tfidf',HashingVectorizer(token_pattern=r'\w{3,}', stop_words='english', alternate_sign = False)),
            ('clf',OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])


model_pipeline3 = Pipeline([
           ('tfidf',HashingVectorizer(token_pattern=r'\w{3,}', stop_words='english',alternate_sign = False)),
           ('clf',ensemble.RandomForestClassifier(n_estimators=10,verbose=3,n_jobs=-1)),])

model_pipeline4 = Pipeline([
          ('tfidf', HashingVectorizer(analyzer='word', token_pattern=r'\w{2,}',stop_words='english',alternate_sign = False)),
           ('clf', OneVsRestClassifier(LogisticRegression(solver='sag',max_iter=500), n_jobs=1)),])

model_pipeline5 = Pipeline([
          ('tfidf', HashingVectorizer(analyzer='word', token_pattern=r'\w{2,}',stop_words='english',alternate_sign = False)),
            ('clf',OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])


model_pipeline6 = Pipeline([
          ('tfidf', HashingVectorizer(analyzer='word', token_pattern=r'\w{2,}',stop_words='english', alternate_sign = False)),
            ('clf',ensemble.RandomForestClassifier(n_estimators=10,verbose=3,n_jobs=-1)),])

model_pipeline7 = Pipeline([
            ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{3,}', stop_words='english',min_df=5)),
            ('clf', OneVsRestClassifier(LogisticRegression(solver='sag',max_iter=500), n_jobs=1)),])

model_pipeline8 = Pipeline([
            ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{3,}', stop_words='english',min_df=5)),
            ('clf',OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])


model_pipeline9 = Pipeline([
('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{3,}', stop_words='english',min_df=5,lowercase=False)),
           ('clf',RandomForestClassifier(verbose=3,n_jobs=-1,class_weight='balanced')),])

model_pipeline10 = Pipeline([
        ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{3,}',ngram_range=(1,2), stop_words='english',min_df=5)),
        ('clf', OneVsRestClassifier(LogisticRegression(solver='sag',max_iter=500), n_jobs=1)),])

model_pipeline11 = Pipeline([
        ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{3,}',ngram_range=(1,2), stop_words='english',min_df=5)),
        ('clf',OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])



model_pipeline12 = Pipeline([
          ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{3,}',ngram_range=(1,2), stop_words='english',min_df=5)),
         ('clf',ensemble.RandomForestClassifier(n_estimators=50,verbose=3,n_jobs=-1)),])

model_pipeline13 = Pipeline([
        ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{11,}',ngram_range=(1,3), stop_words='english',min_df=5)),
        ('clf', OneVsRestClassifier(LogisticRegression(solver='sag',max_iter=500), n_jobs=1)),])

model_pipeline14 = Pipeline([
        ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{11,}',ngram_range=(1,3), stop_words='english',min_df=5)),
            ('clf',OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])


model_pipeline15 = Pipeline([
       ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{11,}',ngram_range=(1,3), stop_words='english',min_df=5)),
            ('clf',ensemble.RandomForestClassifier(n_estimators=10,verbose=3,n_jobs=-1)),])

model_pipeline16 = Pipeline([
      ('tfidf', HashingVectorizer(analyzer='word', token_pattern=r'\w{3,}',ngram_range=(1,3), stop_words='english', alternate_sign = False)),
            ('clf', OneVsRestClassifier(LogisticRegression(solver='sag',max_iter=500), n_jobs=1)),])

model_pipeline17 = Pipeline([
        ('tfidf', HashingVectorizer(analyzer='word', token_pattern=r'\w{3,}',ngram_range=(1,3), stop_words='english',alternate_sign = False)),
            ('clf',OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])


model_pipeline18 = Pipeline([
       ('tfidf', HashingVectorizer(analyzer='word', token_pattern=r'\w{3,}',ngram_range=(1,3), stop_words='english', alternate_sign = False)),
            ('clf',ensemble.RandomForestClassifier(n_estimators=80,verbose=3,n_jobs=-1)),])


In [10]:
model_pipeline = [model_pipeline1, model_pipeline2,model_pipeline3,model_pipeline4,
             model_pipeline5,model_pipeline6,model_pipeline7,model_pipeline8,
             model_pipeline9,model_pipeline10,model_pipeline11,model_pipeline12,
             model_pipeline13,model_pipeline14,model_pipeline15,model_pipeline16,
             model_pipeline17,model_pipeline18]

In [11]:
import string

dataset.dropna(subset=['sentence'], inplace=True)
dataset.reset_index(drop=True, inplace=True)
dataset['cleaned_data'] = dataset['sentence'].apply(lambda x: x.lower())
dataset['cleaned_data'] = dataset['cleaned_data'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
dataset['cleaned_data']= dataset['cleaned_data'].str.findall('\w{2,}').str.join(' ')

In [12]:
dataset.head()

,sentence,flag,cleaned_data
0,One of the other reviewers has mentioned that ...,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,1,wonderful little production br br the filming ...
2,I thought this was a wonderful way to spend ti...,1,thought this was wonderful way to spend time o...
3,Basically there's a family where a little boy ...,0,basically theres family where little boy jake ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love in the time of money is vi...


In [13]:
train_x, test_x, train_y, test_y = train_test_split(dataset['cleaned_data'], dataset['flag'], test_size=0.2, random_state=42)

In [30]:
results_df = pd.DataFrame()
test_y = test_y.astype(int)

for i in range(len(model_pipeline)):
    model_pipeline[i].fit(train_x, train_y)
    print(model_pipeline[i])
    pred = model_pipeline[i].predict_proba(test_x)
    print('--- prediction completed ---')
    y_pred = []
    for row in pred:
        y_pred.append(row[1])
    y_pred = np.array(y_pred)
    prediction = np.where(y_pred > 0.5, 1, 0)
    thres = 0.5
    temp_df = pd.DataFrame()
    temp_df.loc[i,'model_pipeline_number'] = "model_pipeline"+str(i+1)
    temp_df.loc[i,'model_pipeline_spec'] = str(model_pipeline[i])
    temp_df.loc[i,'thres'] = thres
    temp_df.loc[i,'accuracy'] = accuracy_score(test_y, prediction)
    temp_df.loc[i,'recall'] = recall_score(test_y, prediction,average='binary')
    temp_df.loc[i,'precision'] = precision_score(test_y, prediction,average='binary')
    temp_df.loc[i,'f1_score'] = f1_score(test_y, prediction,average='binary')
    results_df = pd.concat([results_df, temp_df], ignore_index=True)
    print('--- output stored in file ---')

Pipeline(steps=[('tfidf',
                 HashingVectorizer(alternate_sign=False, stop_words='english',
                                   token_pattern='\\w{3,}')),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=500,
                                                                  solver='sag'),
                                     n_jobs=1))])
--- prediction completed ---
--- output stored in file ---
Pipeline(steps=[('tfidf',
                 HashingVectorizer(alternate_sign=False, stop_words='english',
                                   token_pattern='\\w{3,}')),
                ('clf', OneVsRestClassifier(estimator=MultinomialNB()))])
--- prediction completed ---
--- output stored in file ---


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 10building tree 2 of 10

building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


Pipeline(steps=[('tfidf',
                 HashingVectorizer(alternate_sign=False, stop_words='english',
                                   token_pattern='\\w{3,}')),
                ('clf',
                 RandomForestClassifier(n_estimators=10, n_jobs=-1,
                                        verbose=3))])


...

[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  6.4min finished


Pipeline(steps=[('tfidf',
                 HashingVectorizer(alternate_sign=False, ngram_range=(1, 3),
                                   stop_words='english',
                                   token_pattern='\\w{3,}')),
                ('clf',
                 RandomForestClassifier(n_estimators=80, n_jobs=-1,
                                        verbose=3))])


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    1.8s


--- prediction completed ---
--- output stored in file ---


[Parallel(n_jobs=2)]: Done  80 out of  80 | elapsed:    5.0s finished


In [31]:
results_df.head()

,model_pipeline_number,model_pipeline_spec,thres,accuracy,recall,precision,f1_score
0,model_pipeline1,"Pipeline(steps=[('tfidf',\n Ha...",0.5,0.8853,0.903155,0.873512,0.888087
1,model_pipeline2,"Pipeline(steps=[('tfidf',\n Ha...",0.5,0.8657,0.851161,0.878533,0.864631
2,model_pipeline3,"Pipeline(steps=[('tfidf',\n Ha...",0.5,0.7553,0.684858,0.800696,0.738261
3,model_pipeline4,"Pipeline(steps=[('tfidf',\n Ha...",0.5,0.8852,0.902560,0.873775,0.887934
4,model_pipeline5,"Pipeline(steps=[('tfidf',\n Ha...",0.5,0.8638,0.847390,0.878059,0.862452


In [32]:
results_df.to_csv('/content/drive/MyDrive/ml_metrics_output.csv', index=False)